<a href="https://colab.research.google.com/github/sachinn1586/ARRAY-AND-FUNCTION-/blob/main/CNN_Optimization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


# Set random seeds for reproducibility
torch.manual_seed(42)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")



Using device: cuda


In [39]:
df = pd.read_csv('/content/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [40]:
df.shape

(6000, 785)

In [41]:
X = df.iloc[:,1:].values
y =df.iloc[:,0].values

In [42]:
X.shape,y.shape

((6000, 784), (6000,))

In [43]:
y

array([9, 7, 0, ..., 8, 4, 8])

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
X_train = X_train/255.0
X_test = X_test/255.0

In [46]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test,y_test)

In [47]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [48]:
class MyNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()

        self.features = nn.Sequential(

            nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)

        return x

In [49]:
learning_rate =0.1
epochs=100

In [50]:

model = MyNN(1)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 1.0533171314001084
Epoch: 2 , Loss: 0.6726930665969849
Epoch: 3 , Loss: 0.577600992123286
Epoch: 4 , Loss: 0.5065669362743696
Epoch: 5 , Loss: 0.47688379655281704
Epoch: 6 , Loss: 0.4407339283823967
Epoch: 7 , Loss: 0.3921127178271612
Epoch: 8 , Loss: 0.3658375396579504
Epoch: 9 , Loss: 0.35916913678248724
Epoch: 10 , Loss: 0.31314054197321334
Epoch: 11 , Loss: 0.30165022214253745
Epoch: 12 , Loss: 0.2930567851414283
Epoch: 13 , Loss: 0.2968810678273439
Epoch: 14 , Loss: 0.23885951561232407
Epoch: 15 , Loss: 0.23996911734342574
Epoch: 16 , Loss: 0.22334616740544638
Epoch: 17 , Loss: 0.21738693987329802
Epoch: 18 , Loss: 0.21621820114552975
Epoch: 19 , Loss: 0.19773231693853935
Epoch: 20 , Loss: 0.16770178655783335
Epoch: 21 , Loss: 0.1751200186088681
Epoch: 22 , Loss: 0.15000387268140913
Epoch: 23 , Loss: 0.15937495308928193
Epoch: 24 , Loss: 0.1523362962435931
Epoch: 25 , Loss: 0.1414071102440357
Epoch: 26 , Loss: 0.1314309312775731
Epoch: 27 , Loss: 0.141286515543858

In [51]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8625


In [52]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.993125
